In [1]:
import pandas as pd
df = pd.read_parquet('./Binary/data/cancer.parquet')
df['mean_rad_gt_mean'] = df['mean_radius'] > df['mean_radius'].mean()
df.head()

,has_cancer,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,...,worst_texture,worst_perimeter,worst_area,worst_smoothness,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension,mean_rad_gt_mean
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,True
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,True
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,True
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,False
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,True


In [2]:
from sbDtypes import ColDType

In [3]:
df.has_cancer

0      1
1      1
2      1
3      1
4      1
      ..
564    1
565    1
566    1
567    1
568    0
Name: has_cancer, Length: 569, dtype: int64

In [4]:
col = ColDType(df.has_cancer)
col

ColDType(has_cancer)

In [5]:
col.s

0      1
1      1
2      1
3      1
4      1
      ..
564    1
565    1
566    1
567    1
568    0
Name: has_cancer, Length: 569, dtype: int64

In [6]:
col.s.drop_duplicates().reset_index(drop=True)

0    1
1    0
Name: has_cancer, dtype: int64

In [7]:
col.get_unique_values() 

0    1
1    0
Name: has_cancer, dtype: int64

In [8]:
col.s.name

'has_cancer'

In [9]:
col.is_type_known

False

In [10]:
# col._is_binary()
col.is_binary

In [11]:
col.is_binary

In [12]:
col.get_unique_values()

0    1
1    0
Name: has_cancer, dtype: int64

In [13]:
col.sb_dtype(return_=True)

3
1


'binary'

In [15]:
col.is_binary

True

In [16]:
col.s_fmt

In [14]:
col.s.drop_duplicates().reset_index(drop=True)

0    1
1    0
Name: has_cancer, dtype: int64